In [1]:
import requests
import pandas as pd
import io
import pickle
import os
# import pymysql
# import boto3
#pd.options.display.max_rows = 999

/Users/jeffshamp/.conda/envs/bxd_etl/lib/python2.7/site-packages/pandas/_libs/__init__.py:3: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from .tslib import iNaT, NaT, Timestamp, Timedelta, OutOfBoundsDatetime
/Users/jeffshamp/.conda/envs/bxd_etl/lib/python2.7/site-packages/pandas/__init__.py:26: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from pandas._libs import (hashtable as _hashtable,
/Users/jeffshamp/.conda/envs/bxd_etl/lib/python2.7/site-packages/pandas/core/dtypes/common.py:6: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from pandas._libs import algos, lib
/Users/jeffshamp/.conda/envs/bxd_etl/lib/python2.7/site-packages/pandas/core/util/hashing.py:7: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from pandas._libs import hashing, tslib
/Users/jeffshamp/.conda

In [6]:
if os.path.exists("OCA_last_modified.txt"):
    with open("OCA_last_modified.txt", "rb") as fp:
        mod_list = pickle.load(fp)
else:
    mod_list = []

ValueError: unsupported pickle protocol: 3

In [3]:
mod_list

NameError: name 'mod_list' is not defined

In [4]:
link_base= "https://www.nycourts.gov/"
link_path = "LegacyPDFS/court-research/OCA-STAT-Act.csv"
link_url = link_base+link_path
link_url

'https://www.nycourts.gov/LegacyPDFS/court-research/OCA-STAT-Act.csv'

In [8]:
def is_downloadable(link_url, mod_list):
    """
    Does the url contain a downloadable resource
    """
    h = requests.head(link_url, allow_redirects=True)
    header = h.headers
    content_type = header.get('content-type')
    if 'text' in content_type.lower():
        return False
    if 'html' in content_type.lower():
        return False
    if header["Last-Modified"] in mod_list:
        return False
    
    return True

In [9]:
if is_downloadable(link_url, mod_list):
    r = requests.get(link_url)
    last_mod = r.headers['Last-Modified']
    mod_list.append(last_mod)
    oca_data = pd.read_csv(io.StringIO(r.text))
else: print("File not downloadable or exists already.")

File not downloadable or exists already.


In [5]:
if mod_list:
    with open("OCA_last_modified.txt", "wb") as fp:
        pickle.dump(mod_list, fp)

In [6]:
mod_list

['Wed, 06 Jan 2021 21:12:51 GMT',
 'Mon, 08 Feb 2021 20:32:30 GMT',
 'Tue, 06 Apr 2021 19:57:13 GMT']

In [10]:
# Re-scrape - Takes a while so use cautiously
re_load = True
if re_load:
    r = requests.get(link_url)
    oca_data = pd.read_csv(io.StringIO(r.text))

In [11]:
oca_main = oca_data.copy()

In [104]:
# Restore from copy
quick_load = True
if quick_load:
    oca_data = oca_main

In [12]:
keep_cols = ['county', 'court', 'arresting_agency', 'arrest_type', 
           'top_charge_at_arraignment', 'severity', 'weight', 'law',
           'article_section', 'attempt_flag', 'docket_status',
           'disposition_type', 'most_severe_sentence', 'article', 'section']
bx_pd = [f"nypd 04{i}" for i in range(10)] + \
        [f"nypd 05{i}" for i in range(3)]

In [13]:
# basic clean up and standardization 
oca_data = oca_data.applymap(lambda x:x.lower() if type(x) == str else x)
oca_data.columns = oca_data.columns.str.replace(" ", "_").str.replace(".", "_")
oca_data.columns = oca_data.columns.str.lower()
oca_data.article_section = oca_data.article_section.str.rstrip(".")
oca_data.article_section = oca_data.article_section.str.replace("\.\.", ".")
oca_data.drop("row_#",axis=1, inplace=True)
oca_data[["article", "section"]] = oca_data.article_section.str.split(".", n=1,expand=True)

In [14]:
from utils import *
oca_data = make_clean_precincts(oca_data)

fel_weights = [x for x in oca_data.weight.unique() if "f" in x]
fel_series = oca_data[oca_data.weight.isin(fel_weights)] \
    .article.apply(lambda x: make_felonies(x))
vio_series = oca_data[oca_data.law.isin(["vtl"])] \
    .article.apply(lambda x: make_violations(x))
bxd_category = pd.concat([fel_series, vio_series]) \
    .rename("bxd_category")
oca_data = oca_data.join(bxd_category)

## Questions for Policy Peeps:

1) What all do we want to track? Misdemeanors _other_ than 511s ?

2) How to determine weapon charge from article/section/weight/law? Our data (I think) uses docket number info to determine this? No docket number in OCA STAT

3) manslaughter vs assault (120/125) - do we want to delineate? 

In [15]:
weapons = ("|").join(["wea", "wpn", "wep"])
oca_data['weapon_charge'] = False
oca_data.loc[(oca_data.top_charge_at_arraignment.str.contains(weapons)) &
    (oca_data.weight != "v"), 'weapon_charge'] = True

sex_charge = ("|").join(["sex", "sx"])
oca_data["sex_charge"] = False
oca_data.loc[oca_data.top_charge_at_arraignment.str.contains(sex_charge),
         "sex_charge"] = True

In [16]:
write_csv = False
if write_csv:
    oca_data.to_csv("oca_data_clean_final.csv")

In [27]:
oca_data.groupby(['article', 'top_charge_at_arraignment']).count()['court'].sort_values(ascending=False)[:10]

article  top_charge_at_arraignment     
120      aslt 3-w/int cause phys injury    9748
155      petit larceny                     7150
220      crim poss contrl subst-7th        4941
215      crim contempt-2nd:disobey crt     4502
0511     aggravated unlic oper veh-3rd     4174
265      cpw-2nd: loaded firearm           3982
1192     op mv while intoxicated 1         3273
145      crim mis:intent damage proprty    3051
120      menacing-2nd:weapon               2131
         aslt w/int cause ph inj w/weap    1901
Name: court, dtype: int64

## Ground to Cloud Upload

And now we wrestle with Jeff Bezos to get this information into our database programmatically. 

### Question and TODO

1) IAM user roles and permission?

2) S3 bucket to store the root file?


3) Action point: Use `pyscopg2` to connect to BxD Redshift instance
    - Check on access keys via enviroment variables for python runtime.

4) Action point: set up "copy" command to desired table from S3. This uses `psycopg2` as well. 
    - Check on auto commit functionality for this COPY command. 

### Prototyping

Possible commands for automated upload to AWS. 

Use `boto3` to upload to S3.

In [ ]:
import boto3

file_path = "<<-path_to_file->>"
bucket_name = "<<-BxD_bucket->>"

def aws_session(region_name='us-east-1'):
    return boto3.session.Session(
        # use env variables to store AWS keys
        # BxD might do something else with this. 
        # currently, this is a placeholder
        aws_access_key_id=os.getenv('AWS_ACCESS_KEY_ID'),
        aws_secret_access_key=os.getenv('AWS_ACCESS_KEY_SECRET'),
        region_name=region_name
    )

def upload_file_to_bucket(bucket_name, file_path):
    session = aws_session()
    s3_resource = session.resource('s3')
    file_dir, file_name = os.path.split(file_path)

    bucket = s3_resource.Bucket(bucket_name)
    bucket.upload_file(
      Filename=file_path,
      Key=file_name,
      # ExtraArgs={'ACL': 'public-read'} probably don't want this 
      # Someone should know what the access control list should be (Jesus)
    )
    
    # below is optional for confirmation that things when to the right place. 
    s3_url = f"https://{bucket_name}.s3.amazonaws.com/{file_name}"
    return s3_url

Run the functions!

In [ ]:
session = aws_session()
s3_bucket_url = upload_file_to_bucket(bucket_name, file_path)

Use `psycopg2` to copy csv file in S3 to BxD DB in Redshift

In [ ]:
import psycopg2

def redshift():

    conn = psycopg2.connect(dbname='**BxD_DB**',
                            host='888888888888****.u.****.redshift.amazonaws.com',
                            port='****', 
                            user='******', 
                            password='********')
    cur = conn.cursor();
    
    # We can use env variables here as well with f-string literals if we want.
    command = "copy <<-OCA_table_name->> from "
    bucket_url = "s3://<<-path_to_file_s3->>.csv "
    creds = "credentials 'aws_access_key_id=<<-ID->>;aws_secret_access_key=<<-KEY->>' csv;"
    copy_command = command+bucket_url+creds
    
    # Begin your transaction
    cur.execute("begin;")

    cur.execute(copy_command)
    
    # Commit your transaction if auto commits are not enabled
    cur.execute("commit;")
    
    # print("Copy executed fine!")

Run it

In [ ]:
redshift()